<a href="https://www.kaggle.com/code/madeeltariq/api-gateway-rag?scriptVersionId=214156681" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# *Installing the required libraries*# 

In [1]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("transformers")
install_package("pinecone-client")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")


flask is already installed.
transformers is already installed.
Installing pinecone-client...
pinecone-client installation completed.
Installing python-dotenv...
python-dotenv installation completed.
Installing flask-cors...
flask-cors installation completed.
Installing pyngrok...
pyngrok installation completed.
ngrok installed successfully.
Done With All installations >>>>>>>>>>>>>>>>> Move Forward


# Imports in here

## **Code cell for Entering the ngrok API key For Ngrok implementation**

In [2]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")

# Function to create/update .env file with Pinecone credentials
def set_pinecone_credentials():
    # Ask user for the Pinecone API key without displaying the input
    pinecone_api_key = getpass.getpass("Please enter your Pinecone API key: ")

    # Ask user for the Pinecone environment, with a default value
    pinecone_env = input("Please enter your Pinecone environment (e.g., us-west1-gcp) [default: us-east-1]: ") or "us-east-1"
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the keys; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    
    print(f"Pinecone credentials added to {env_file}")

# Call the functions to set the API key and credentials
set_ngrok_api_key()
set_pinecone_credentials()


Please enter your ngrok API key:  ········


ngrok API key added to .env


Please enter your Pinecone API key:  ········
Please enter your Pinecone environment (e.g., us-west1-gcp) [default: us-east-1]:  


Pinecone credentials added to .env


In [8]:
import os
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from pinecone import Pinecone

# Function to get vector from Pinecone database based on ID
def get_vector_from_pinecone(file_id, index_name="vecotr", namespace_name="newCheck", pinecone_env="us-east-1"):
    # Load environment variables from .env file
    load_dotenv()  # This will load the variables from the .env file into the environment

    # Load Pinecone API key from the environment variables
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        print("Check if ENV variables are set")

    # Ensure the API key is not None
    if not pinecone_api_key:
        # print("Error: Pinecone API key is missing from the environment variables.")
        return None

    # Initialize Pinecone client
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        print(f"Pinecone client initialized with environment: {pinecone_env}")
    except Exception as e:
        print(f"Error initializing Pinecone client: {str(e)}")
        return None

    # Initialize the index
    try:
        index = pc.Index(index_name)
    except Exception as e:
        print(f"Error initializing Pinecone index: {str(e)}")
        return None

    # Query Pinecone to retrieve vector for the specific ID
    try:
        print(f"Retrieving vector for file ID: {file_id} from Pinecone...")

        # Querying Pinecone index for the specific ID using metadata filtering
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            id=file_id,
            top_k=1,
            include_values=True,
            include_metadata=True
        )

        # Check if result contains matches
        if response and response.get("matches"):
            # Display the first few vectors based on top_k
            for idx, vector_data in enumerate(response["matches"]):
                # print(f"Vector {idx + 1} ID: {vector_data['id']}")
                print(f"Pinecone Index: {index_name}")
                print(f"Pinecone Env: {pinecone_env}")
                # print(f"Metadata (fileId): {vector_data['metadata']['fileId']}")
                # print(f"Metadata (text): {vector_data['metadata']['text'][:100]}...")  # Showing first 100 chars of text
                print(f"Vector Values: {vector_data['values'][:5]}...")  # Showing first 5 values of the vector
                print(f"Size of Vector Values: {len(vector_data['values'])}")  # Printing the size of the vector values
            return response["matches"]

        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving vector for file ID {file_id}: {str(e)}")
        return None

# Example usage of the function
file_id = "676594520c3eb8c3272efa2c"  # Replace with the file ID you want to query
index_name = "vecotr"  # The Pinecone index name
namespace_name = "newCheck"  # The namespace in Pinecone
pinecone_env = "us-east-1"  # The Pinecone environment

vector = get_vector_from_pinecone(file_id, index_name=index_name, namespace_name=namespace_name, pinecone_env=pinecone_env)



Pinecone client initialized with environment: us-east-1
Retrieving vector for file ID: 676594520c3eb8c3272efa2c from Pinecone...
Pinecone Index: vecotr
Pinecone Env: us-east-1
Vector Values: [0.029296875, -0.00984191895, -0.0322265625, -0.0369262695, 0.0473632812]...
Size of Vector Values: 1024


# To test this one is 

In [29]:
import os
from dotenv import load_dotenv  # For loading .env file
from pinecone import Pinecone
from transformers import pipeline

# Load environment variables from .env file
load_dotenv()

# Initialize the Pinecone API key and environment from .env
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV', 'us-east-1')

# Initialize Pinecone client
def initialize_pinecone():
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        print(f"Pinecone client initialized with environment: {pinecone_env}")
        return pc
    except Exception as e:
        print(f"Error initializing Pinecone client: {str(e)}")
        return None

# Retrieve context from Pinecone
def get_context_from_pinecone(file_id, index_name="vecotr", namespace_name="newCheck"):
    try:
        pc = initialize_pinecone()
        if not pc:
            return None

        index = pc.Index(index_name)

        # Query Pinecone to retrieve vector for the specific ID
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            id=file_id,
            top_k=1,
            include_values=True,
            include_metadata=True
        )

        if response and response.get("matches"):
            context = response["matches"][0]['metadata']['text']  # Assuming the text is stored in metadata
            print(f"Context for file {file_id}: {context[:100]}...")  # Display first 100 characters
            return context
        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving context for file ID {file_id}: {str(e)}")
        return None

# Initialize the GPT-J model from Hugging Face
def initialize_model():
    generator = pipeline("text-generation", model="EleutherAI/gpt-j-6B")
    return generator

# Generate response using the model and context
def generate_response(user_query, context, model):
    prompt = f"Question: {user_query}\nContext: {context}\nAnswer:"
    print(f"Prompt: {prompt[:200]}...")  # Displaying first 200 chars of the prompt for debugging
    response = model(prompt, max_length=150, num_return_sequences=1)
    return response[0]['generated_text']

# Function to maintain the session
def chat_with_model(file_id, user_query):
    context = get_context_from_pinecone(file_id)
    if not context:
        return "Sorry, I couldn't retrieve the relevant context."

    model = initialize_model()
    
    # Generate the response based on the user query and context
    response = generate_response(user_query, context, model)
    
    return response

# Example usage: The user asks a question and the system provides an answer
file_id = "676594520c3eb8c3272efa2c"  # Replace with the file ID you want to query
user_query = "Can you explain what Compiler Construction is?"
response = chat_with_model(file_id, user_query)

print("Response:", response)


Pinecone client initialized with environment: us-east-1
Context for file 676594520c3eb8c3272efa2c: 

Page 1 of 1 
 
 
 
DEPARTMENT OF COMPUTER SCIENCE 
UNIVERSITY OF ENGINEERING AND TECHNOLOGY, NEW C...


config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Question: Can you explain what Compiler Construction is?
Context: 

Page 1 of 1 
 
 
 
DEPARTMENT OF COMPUTER SCIENCE 
UNIVERSITY OF ENGINEERING AND TECHNOLOGY, NEW CAMPUS 
CS-471L Compiler Constructi...


ValueError: Input length of input_ids is 150, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

# Implemented Ngrok and CORS on top of the Flask API

In [1]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import dotenv to load .env file

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Health check endpoint with request body and logging
@app.route('/health', methods=['GET', 'POST'])
def health_check():
    try:
        # Log the request data and ID (if present)
        if request.method == 'POST':
            data = request.json
            print("Received request data:", data)
            id_value = data.get("id", "No ID provided")
            print("Received ID:", id_value)
        else:
            print("Received request data:", request.args)
        
        # Respond with a Good message
        return jsonify({"status": "Flask server is running great!"})
    except Exception as e:
        # Catch any exceptions and respond with an error message
        print(f"Error: {str(e)}")
        return jsonify({"status": "Error occurred while processing the request."}), 500

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 5000...")
    app.run(host="0.0.0.0", port=5000)  # Running on port 5000

# Function to start ngrok and expose the Flask server
def run_ngrok():
    # Authenticate ngrok using the API key from the .env file
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
        print("Ngrok authenticated successfully.")
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    # Open a ngrok tunnel to the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Start ngrok and print the URL where Flask is accessible
public_url = run_ngrok()

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


Flask server is listening on port 5000...
 * Serving Flask app '__main__'
 * Debug mode: off
Ngrok authenticated successfully.
Ngrok public URL: NgrokTunnel: "https://c89e-34-71-204-254.ngrok-free.app" -> "http://localhost:5000"
Received request data: {'id': 12345}
Received ID: 12345
Received request data: {'id': 12345, 'url': 'httpsasdfadsf'}
Received ID: 12345


KeyboardInterrupt: 

# *Working Prototype for flask listening***

In [ ]:
import os
from flask import Flask, request, jsonify
import threading
import time

# Initialize Flask app
app = Flask(__name__)

# Health check endpoint
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "Flask server is running Greate!"})

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 8000...")
    app.run(host="0.0.0.0", port=9000)  # Running on port 8000

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Print out the URL where Flask is running
print("Flask server should be accessible at: http://0.0.0.0:8000")

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


# How to send request from the JavaScript Server Side 

In [ ]:
async function checkFlaskHealth() {
    try {
        // Replace with the ngrok URL provided by the Flask app
        const apiUrl = 'https://c89e-34-71-204-254.ngrok-free.app/health';

        // Prepare the request body with an 'id'
        const requestBody = {
            id: 12345, // Example ID, you can change this to anything
            url: "httpsasdfadsf"
        };

        // Make a POST request to the /health endpoint with the request body
        const response = await fetch(apiUrl, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify(requestBody) // Send the request body as JSON
        });

        // Parse the JSON response
        const data = await response.json();

        // Log the response to the console
        console.log("Response from Flask server:", data);
        
        // Check the status of the response
        if (data.status === "Flask server is running great!") {
            console.log("Flask server is up and running!");
        } else {
            console.log("Flask server is down or returned an unexpected response.");
        }
    } catch (error) {
        console.error("Error while making the request:", error);
    }
}

// Call the function to check the Flask health endpoint
checkFlaskHealth();
